<a href="https://colab.research.google.com/github/ChaaruVijay/DSPL-CourseWork/blob/main/XGB_Boost_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Building Predictive Models - XGB Model**

In [9]:
!pip install xgboost
!pip install scikit-learn

In [10]:
#Loading the dataset
import pandas as pd
import numpy as ny
data = pd.read_csv('/content/train_cleaned.csv')

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 774055 entries, 0 to 774054
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   outlet_city       774055 non-null  int64  
 1   luxury_sales      774055 non-null  float64
 2   fresh_sales       774055 non-null  float64
 3   dry_sales         774055 non-null  float64
 4   cluster_catgeory  774055 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 29.5 MB


In [13]:
#Splitting train.csv into training and testing
from sklearn.model_selection import train_test_split



y = data['cluster_catgeory']

# Split into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

NameError: name 'X' is not defined

In [ ]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [ ]:
print("Unique values in y_train:", y_train.unique())

Unique values in y_train: [6 4 1 2 5 3]


In [ ]:
#Since the model expecs all to be start from 0
y_train = y_train - 1
y_test = y_test - 1
print("Unique values in y_train:", y_train.unique())

Unique values in y_train: [5 3 0 1 4 2]


**Model Development**

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'subsample': [0.8, 1.0],
}

grid_search = GridSearchCV(estimator=xgb.XGBClassifier(objective='multi:softmax', num_class=len(y.unique())),
                           param_grid=param_grid,
                           cv=3,
                           scoring='accuracy',
                           verbose=1)

grid_search.fit(X_train, y_train)

# Best parameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate best model
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f"Best Model Accuracy: {accuracy_best * 100:.2f}%")



Fitting 3 folds for each of 54 candidates, totalling 162 fits
Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Best Model Accuracy: 100.00%
